In [1]:
# Importar as bibliotecas necessárias
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from google.colab import files
import joblib
import zipfile

# Fazer o upload do dataset JSON
uploaded = files.upload()

# Carregar o dataset a partir do arquivo JSON
with open('dataset.json', 'r') as f:
    dataset = json.load(f)

# Converter o dataset para um DataFrame para manipulação mais fácil
df = pd.DataFrame(dataset)

# Limpar dados removendo registros sem letra e sem tópicos
df = df[(df['lyric'].str.strip() != '') & (df['keys'].apply(lambda x: len(x) > 0))]

print(f"Total de registros após limpeza: {len(df)}")

# Passo 1: Separar o dataset em treino e teste antes de qualquer processamento adicional
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Resetar os índices para evitar problemas de desalinhamento
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Passo 2: Configurar a vetorização TF-IDF usando apenas os dados de treinamento
vectorizer = TfidfVectorizer(max_features=5000)
X_train_full = vectorizer.fit_transform(df_train['lyric'])
X_test_full = vectorizer.transform(df_test['lyric'])

# Salvar o vetorizer para uso posterior
joblib.dump(vectorizer, 'vectorizer_tfidf.pkl')

# Passo 3: Criar um dicionário para armazenar os dados de treinamento para cada tópico
topicos = df['keys'].explode().str.strip().unique()
dados_treino = {topico: {'X_train': [], 'y_train': [], 'X_test': [], 'y_test': []} for topico in topicos}

# Passo 4: Preencher o dicionário com os dados correspondentes para cada tópico
for i, row in df_train.iterrows():
    for topico in row['keys']:
        topico = topico.strip()
        dados_treino[topico]['X_train'].append(X_train_full[i].toarray()[0])
        dados_treino[topico]['y_train'].append(1)  # 1 indica que o tópico está presente

# Adicionar exemplos negativos (onde o tópico não está presente) para o conjunto de treinamento
for topico, data in dados_treino.items():
    indices_negativos = [i for i in range(len(df_train)) if topico not in df_train.iloc[i]['keys']]
    for idx in indices_negativos:
        data['X_train'].append(X_train_full[idx].toarray()[0])
        data['y_train'].append(0)  # 0 indica que o tópico não está presente

# Preencher o conjunto de teste com os dados correspondentes
for i, row in df_test.iterrows():
    for topico in row['keys']:
        topico = topico.strip()
        dados_treino[topico]['X_test'].append(X_test_full[i].toarray()[0])
        dados_treino[topico]['y_test'].append(1)

# Adicionar exemplos negativos para o conjunto de teste
for topico, data in dados_treino.items():
    indices_negativos = [i for i in range(len(df_test)) if topico not in df_test.iloc[i]['keys']]
    for idx in indices_negativos:
        data['X_test'].append(X_test_full[idx].toarray()[0])
        data['y_test'].append(0)


Saving dataset.json to dataset.json
Total de registros após limpeza: 100


In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import os

# Criar diretório para salvar os modelos
os.makedirs('modelos_naive_bayes', exist_ok=True)
os.makedirs('resultados', exist_ok=True)

matriz_confusao_total = pd.DataFrame()

# Treinar um modelo de Naive Bayes para cada tópico
for topico, data in dados_treino.items():
    # Verificar se há dados suficientes para treinar o modelo
    if len(data['X_train']) == 0 or len(data['y_train']) == 0:
        print(f'Não há dados para treinar o modelo do tópico "{topico}". Pulando...')
        continue

    # Criar e treinar o modelo
    modelo = MultinomialNB()
    modelo.fit(data['X_train'], data['y_train'])

    # Salvar o modelo treinado com o nome do tópico
    nome_modelo = f'modelos_naive_bayes/modelo_{topico}.pkl'
    joblib.dump(modelo, nome_modelo)

    print(f'Modelo para o tópico "{topico}" treinado e salvo como {nome_modelo}.')

    # Avaliar o modelo usando os dados de teste
    if len(data['X_test']) > 0 and len(data['y_test']) > 0:
        y_pred = modelo.predict(data['X_test'])
        acuracia = accuracy_score(data['y_test'], y_pred)
        print(f'Acurácia do modelo "{topico}": {acuracia:.2f}')

        # Gerar a matriz de confusão
        matriz_confusao = confusion_matrix(data['y_test'], y_pred)

        # Verificar o tamanho da matriz de confusão e ajustar os índices e colunas dinamicamente
        if matriz_confusao.shape == (1, 1):
            # Se for uma matriz 1x1, significa que há apenas uma classe presente (ou tudo verdadeiro ou falso)
            df_matriz_confusao = pd.DataFrame(matriz_confusao,
                                              index=[f'True {topico}'],
                                              columns=[f'Pred {topico}'])
        elif matriz_confusao.shape == (2, 2):
            # Caso normal de uma matriz de confusão 2x2
            df_matriz_confusao = pd.DataFrame(matriz_confusao,
                                              index=[f'True {topico}', f'False {topico}'],
                                              columns=[f'Pred {topico}', f'Not Pred {topico}'])
        else:
            print(f'Matriz de confusão para o tópico "{topico}" possui um formato inesperado: {matriz_confusao.shape}')
            continue  # Se o formato não for esperado, pule este tópico

# Concatenar a matriz de confusão ao DataFrame total
matriz_confusao_total = pd.concat([matriz_confusao_total, df_matriz_confusao], axis=1)


# Salvar a matriz de confusão total em CSV
matriz_confusao_total.to_csv('resultados/matriz_confusao_total.csv', index=True)



Modelo para o tópico "Caminho" treinado e salvo como modelos_naive_bayes/modelo_Caminho.pkl.
Acurácia do modelo "Caminho": 0.70
Modelo para o tópico "Paz" treinado e salvo como modelos_naive_bayes/modelo_Paz.pkl.
Acurácia do modelo "Paz": 0.60
Modelo para o tópico "Alegria" treinado e salvo como modelos_naive_bayes/modelo_Alegria.pkl.
Acurácia do modelo "Alegria": 0.50
Modelo para o tópico "Coração" treinado e salvo como modelos_naive_bayes/modelo_Coração.pkl.
Acurácia do modelo "Coração": 0.85
Modelo para o tópico "Amor" treinado e salvo como modelos_naive_bayes/modelo_Amor.pkl.
Acurácia do modelo "Amor": 0.70
Modelo para o tópico "Irmãos" treinado e salvo como modelos_naive_bayes/modelo_Irmãos.pkl.
Acurácia do modelo "Irmãos": 0.90
Modelo para o tópico "Jesus" treinado e salvo como modelos_naive_bayes/modelo_Jesus.pkl.
Acurácia do modelo "Jesus": 0.65
Modelo para o tópico "Louvor a Deus" treinado e salvo como modelos_naive_bayes/modelo_Louvor a Deus.pkl.
Acurácia do modelo "Louvor a 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use th

Modelo para o tópico "Morte" treinado e salvo como modelos_naive_bayes/modelo_Morte.pkl.
Acurácia do modelo "Morte": 1.00
Modelo para o tópico "Comunidade" treinado e salvo como modelos_naive_bayes/modelo_Comunidade.pkl.
Acurácia do modelo "Comunidade": 0.90
Modelo para o tópico "Misericórdia" treinado e salvo como modelos_naive_bayes/modelo_Misericórdia.pkl.
Acurácia do modelo "Misericórdia": 0.70
Modelo para o tópico "Altar" treinado e salvo como modelos_naive_bayes/modelo_Altar.pkl.
Acurácia do modelo "Altar": 1.00
Modelo para o tópico "Igreja" treinado e salvo como modelos_naive_bayes/modelo_Igreja.pkl.
Acurácia do modelo "Igreja": 0.90
Modelo para o tópico "Perdão" treinado e salvo como modelos_naive_bayes/modelo_Perdão.pkl.
Acurácia do modelo "Perdão": 0.85
Modelo para o tópico "Eucaristia" treinado e salvo como modelos_naive_bayes/modelo_Eucaristia.pkl.
Acurácia do modelo "Eucaristia": 0.80
Modelo para o tópico "Pão" treinado e salvo como modelos_naive_bayes/modelo_Pão.pkl.
Acur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use th

Modelo para o tópico "Luz" treinado e salvo como modelos_naive_bayes/modelo_Luz.pkl.
Acurácia do modelo "Luz": 0.80
Modelo para o tópico "Concórdia" treinado e salvo como modelos_naive_bayes/modelo_Concórdia.pkl.
Acurácia do modelo "Concórdia": 1.00
Modelo para o tópico "Gratidão" treinado e salvo como modelos_naive_bayes/modelo_Gratidão.pkl.
Acurácia do modelo "Gratidão": 1.00
Modelo para o tópico "Discernimento" treinado e salvo como modelos_naive_bayes/modelo_Discernimento.pkl.
Acurácia do modelo "Discernimento": 1.00
Modelo para o tópico "Libertação" treinado e salvo como modelos_naive_bayes/modelo_Libertação.pkl.
Acurácia do modelo "Libertação": 0.85
Modelo para o tópico "Ressurreição" treinado e salvo como modelos_naive_bayes/modelo_Ressurreição.pkl.
Acurácia do modelo "Ressurreição": 0.70
Modelo para o tópico "Deus" treinado e salvo como modelos_naive_bayes/modelo_Deus.pkl.
Acurácia do modelo "Deus": 0.85
Modelo para o tópico "Céu" treinado e salvo como modelos_naive_bayes/model

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use th

Modelo para o tópico "" treinado e salvo como modelos_naive_bayes/modelo_.pkl.
Acurácia do modelo "": 0.95
Modelo para o tópico "Justo" treinado e salvo como modelos_naive_bayes/modelo_Justo.pkl.
Acurácia do modelo "Justo": 1.00
Modelo para o tópico "Animais" treinado e salvo como modelos_naive_bayes/modelo_Animais.pkl.
Acurácia do modelo "Animais": 1.00
Modelo para o tópico "Cruz" treinado e salvo como modelos_naive_bayes/modelo_Cruz.pkl.
Acurácia do modelo "Cruz": 1.00
Modelo para o tópico "Renovo" treinado e salvo como modelos_naive_bayes/modelo_Renovo.pkl.
Acurácia do modelo "Renovo": 1.00
Modelo para o tópico "Confissão" treinado e salvo como modelos_naive_bayes/modelo_Confissão.pkl.
Acurácia do modelo "Confissão": 1.00
Modelo para o tópico "Arrependimento" treinado e salvo como modelos_naive_bayes/modelo_Arrependimento.pkl.
Acurácia do modelo "Arrependimento": 1.00
Modelo para o tópico "Caridade" treinado e salvo como modelos_naive_bayes/modelo_Caridade.pkl.
Acurácia do modelo "C

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use th

Modelo para o tópico "Filho de Deus" treinado e salvo como modelos_naive_bayes/modelo_Filho de Deus.pkl.
Acurácia do modelo "Filho de Deus": 1.00
Modelo para o tópico "Banquete messiânico" treinado e salvo como modelos_naive_bayes/modelo_Banquete messiânico.pkl.
Acurácia do modelo "Banquete messiânico": 1.00
Modelo para o tópico "Autoridade" treinado e salvo como modelos_naive_bayes/modelo_Autoridade.pkl.
Acurácia do modelo "Autoridade": 1.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
!zip -r modelo_naive_bayes.zip ./modelos_naive_bayes ./vectorizer_tfidf.pkl

In [11]:
import os
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score

# Configurar pandas para exibir todos os dados
pd.set_option('display.max_rows', None)  # Exibir todas as linhas
pd.set_option('display.max_columns', None)  # Exibir todas as colunas
pd.set_option('display.expand_frame_repr', False)  # Não quebrar linha

def avaliar_modelo_final(df_test, vectorizer, dados_treino):
    # Criar uma lista para armazenar os resultados da avaliação de cada modelo
    resultados_finais = []

    # Criar a pasta 'resultados' se ela não existir
    os.makedirs('resultados', exist_ok=True)

    # Vetorizar todos os dados de teste
    X_todas = vectorizer.transform(df_test['lyric'])
    y_true_all = df_test['keys'].tolist()  # Verdadeiros valores das tags de cada canto

    # Listas para armazenar as predições e valores reais de todos os tópicos combinados
    y_pred_geral = []
    y_true_geral = []

    # Avaliar cada modelo no conjunto de teste
    for topico in dados_treino.keys():
        modelo = joblib.load(f'modelos_naive_bayes/modelo_{topico}.pkl')
        y_pred = modelo.predict(X_todas)

        # Calcular a acurácia para o tópico
        y_true_binary = [1 if topico in keys else 0 for keys in y_true_all]
        accuracy = accuracy_score(y_true_binary, y_pred)
        report = classification_report(y_true_binary, y_pred, zero_division=0)

        resultados_finais.append({
            'Tópico': topico,
            'Acurácia': accuracy,
            'Relatório de Classificação': report
        })

        # Adicionar os resultados às listas para avaliação completa do modelo
        y_pred_geral.extend(y_pred)
        y_true_geral.extend(y_true_binary)

    # Avaliação completa do modelo combinando todos os tópicos
    acuracia_geral = accuracy_score(y_true_geral, y_pred_geral)
    precisao_geral = precision_score(y_true_geral, y_pred_geral, average='weighted', zero_division=0)
    recall_geral = recall_score(y_true_geral, y_pred_geral, average='weighted', zero_division=0)
    f1_geral = f1_score(y_true_geral, y_pred_geral, average='weighted', zero_division=0)
    relatorio_classificacao_geral = classification_report(y_true_geral, y_pred_geral, zero_division=0)

    # Adicionar a avaliação completa ao DataFrame de resultados finais
    resultados_finais.append({
        'Tópico': 'Avaliação Completa',
        'Acurácia': acuracia_geral,
        'Precisão': precisao_geral,
        'Recall': recall_geral,
        'F1-Score': f1_geral,
        'Relatório de Classificação': relatorio_classificacao_geral
    })

    # Criar um DataFrame com os resultados finais
    df_resultados_finais = pd.DataFrame(resultados_finais)

    # Salvar os resultados finais em um arquivo CSV na pasta 'resultados'
    df_resultados_finais.to_csv('resultados/resultados_finais.csv', index=False)

    return df_resultados_finais

# Chamar a função para avaliar o modelo final
df_resultados_finais = avaliar_modelo_final(df_test, vectorizer, dados_treino)

# Exibir os resultados finais
print(df_resultados_finais)
!zip -r resultado_naive_bayes.zip ./resultados


                  Tópico  Acurácia                         Relatório de Classificação  Precisão    Recall  F1-Score
0                Caminho  0.700000                precision    recall  f1-score   ...       NaN       NaN       NaN
1                    Paz  0.600000                precision    recall  f1-score   ...       NaN       NaN       NaN
2                Alegria  0.500000                precision    recall  f1-score   ...       NaN       NaN       NaN
3                Coração  0.850000                precision    recall  f1-score   ...       NaN       NaN       NaN
4                   Amor  0.700000                precision    recall  f1-score   ...       NaN       NaN       NaN
5                 Irmãos  0.900000                precision    recall  f1-score   ...       NaN       NaN       NaN
6                  Jesus  0.650000                precision    recall  f1-score   ...       NaN       NaN       NaN
7          Louvor a Deus  0.700000                precision    recall  f

In [23]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Carregar o tokenizador
vectorizer = joblib.load('/content/vectorizer_tfidf.pkl')

def classificar_musica():
    musica = input("Digite a letra do canto para classificação: ")

    X_musica = vectorizer.transform([musica])

    topicos_identificados = []

    # limiar de probabilidade
    limiar_probabilidade = 0.2

    # Iterar sobre os modelos treinados para classificar a música
    for topico in dados_treino.keys():
        modelo = joblib.load(f'modelos_naive_bayes/modelo_{topico}.pkl')

        # Verificar se o modelo é capaz de prever probabilidades para ambas as classes
        if hasattr(modelo, "predict_proba"):
            probas = modelo.predict_proba(X_musica)

            # Garantir que o modelo retorne probabilidades para as duas classes
            if probas.shape[1] > 1:
                probabilidade = probas[0][1]
            else:
                # Se só houver uma classe prevista, definir a probabilidade para 0 ou 1
                probabilidade = 1.0 if modelo.predict(X_musica)[0] == 1 else 0.0
        else:
            # Se o modelo não suportar predict_proba, usar predict diretamente
            probabilidade = 1.0 if modelo.predict(X_musica)[0] == 1 else 0.0

        # Se a probabilidade for maior que o limiar, considerar o tópico relevante
        if probabilidade >= limiar_probabilidade:
            topicos_identificados.append(topico)

    if topicos_identificados:
        print("Tópicos bíblicos identificados para este canto:")
        for topico in topicos_identificados:
            print(f"- {topico}")
    else:
        print("Nenhum tópico bíblico foi identificado para este canto.")

classificar_musica()


Digite a letra do canto para classificação: Na mesa sagrada se faz unidade No pão que alimenta, que é o pão do senhor Formamos família na fraternidade Não há diferença de raça ou de cor Importa viver, senhor, unidos no amor Na participação, vivendo em comunhão. Chegar junto à mesa é comprometer-seÉ a deus converter-se com sinceridadeO grito dos fracos devemos ouvir E em nome de cristo amar e servir! Enquanto na terra o pão for partido O homem nutrido se transformará Vivendo a esperança num mundo melhorCom cristo lutando, o amor vencerá. Se participamos da eucaristiaÉ grande a alegria que deus oferece Porém não podemos deixar esquecida A dor, nesta vida, que o pobre padece. Assim, comungando da única vidaA morte vencida será nossa sorteSe unidos buscarmos libertação Teremos com cristo a ressurreição
Tópicos bíblicos identificados para este canto:
- Amor
- Esperança
- Comunhão
